In [1]:
# importing required libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.ensemble import StackingRegressor

In [2]:
train_data= pd.read_csv('processed_train.csv',index_col=0)

In [3]:
y = train_data.pop('cltv')
X = train_data.copy()

In [4]:
# Split the data into 70% training, 15% evaluation, and 15% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print(f'X_train shape : {X_train.shape}')
print(f'X_test shape : {X_test.shape}')

X_train shape : (71492, 22)
X_test shape : (17874, 22)


## Model-1 Linear Model

In [6]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)
print(f'linear model r2 score is {linear_model.score(X_test,y_test)}')

linear model r2 score is 0.15841545929133272


In [7]:
# saving results
test_data = pd.read_csv('processed_test.csv',index_col=0)
prediction1= linear_model.predict(test_data)

submission = pd.DataFrame({'id':test_data.index,'cltv':prediction1})

submission.to_csv('submission_linear_model.csv',index=False)

## Model 2- Catboost

In [8]:


# initialize the model
cat_model = CatBoostRegressor(iterations=1200, learning_rate=0.01, depth=6,loss_function='RMSE')

# fit the model on the training data
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), logging_level='Silent',plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [12]:

# make predictions on the test set
y_pred_cat = cat_model.predict(X_test)
print(f'r2 score of catboost model {r2_score(y_test, y_pred_cat)}')

r2 score of catboost model 0.16381235169549657


High score is due to we test dataset as evaluation parameter above

In [13]:
# seeing important features
feature_importance= pd.DataFrame({'feature':X_train.columns,'importance':cat_model.feature_importances_})
feature_importance.sort_values(by='importance',ascending=False)[:10].reset_index(drop=['id'])

,feature,importance
0,num_policies_More than 1,72.296026
1,claim_amount,4.034353
2,claim_per_policy,3.710989
3,purchasing_power,2.296248
4,marital_status_not married,2.288515
5,policy_B,2.211260
6,area_Urban,2.093226
7,claim_amount_per_year,2.032714
8,policy_C,1.805925
9,earning_to_company,1.666762


In [14]:
# saving results
test_data = pd.read_csv('processed_test.csv',index_col=0)

prediction2= cat_model.predict(test_data)

submission = pd.DataFrame({'id':test_data.index,'cltv':prediction2})

submission.to_csv('submission_cat_model.csv',index=False)

This resulted in test_data r2 score = 0.1590

## Model 3 XGBRF Regressor

In [15]:
model_xgbrf_regressor = xgb.XGBRFRegressor()
model_xgbrf_regressor.fit(X_train,y_train)

XGBRFRegressor(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
               grow_policy='depthwise', importance_type=None,
               interaction_constraints='', max_bin=256, max_cat_threshold=64,
               max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=0, num_parallel_tree=100,
               objective='reg:squarederror', predictor='auto', random_state=0,
               reg_alpha=0, ...)

In [16]:
# make predictions on the test set
y_pred_xgbrf_regressor = model_xgbrf_regressor.predict(X_test)

# evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgbrf_regressor))
print("RMSE: ", rmse)
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred_xgbrf_regressor))

RMSE:  82935.62828004149
0.16025082420607417


In [17]:
# saving results
test_data = pd.read_csv('processed_test.csv',index_col=0)
prediction3= model_xgbrf_regressor.predict(test_data)

submission = pd.DataFrame({'id':test_data.index,'cltv':prediction3})

submission.to_csv('submission_xgbrf.csv',index=False)

accuracy on test data 0.155797714687998

## Model-4 Stacking of different models

In [18]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

estimators = [('lr', LinearRegression()), 
              ('catboost', CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6,loss_function='RMSE')),
              ('xgbrf',xgb.XGBRFRegressor())]

stacked_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacked_model.fit(X_train, y_train)

0:	learn: 90515.3991451	total: 17.3ms	remaining: 17.3s
1:	learn: 90378.4341637	total: 36.4ms	remaining: 18.2s
2:	learn: 90245.8896688	total: 52.1ms	remaining: 17.3s
3:	learn: 90116.0855674	total: 60.5ms	remaining: 15.1s
4:	learn: 89987.3185061	total: 69.9ms	remaining: 13.9s
5:	learn: 89863.3681413	total: 77.4ms	remaining: 12.8s
6:	learn: 89740.1202862	total: 86.8ms	remaining: 12.3s
7:	learn: 89620.2310250	total: 94.7ms	remaining: 11.7s
8:	learn: 89501.0400317	total: 102ms	remaining: 11.3s
9:	learn: 89383.9788390	total: 110ms	remaining: 10.9s
10:	learn: 89270.9805791	total: 117ms	remaining: 10.5s
11:	learn: 89158.2671918	total: 126ms	remaining: 10.4s
12:	learn: 89047.2064222	total: 134ms	remaining: 10.2s
13:	learn: 88941.4188821	total: 143ms	remaining: 10.1s
14:	learn: 88835.5993145	total: 151ms	remaining: 9.92s
15:	learn: 88732.0769223	total: 159ms	remaining: 9.8s
16:	learn: 88630.2054980	total: 167ms	remaining: 9.68s
17:	learn: 88529.1709223	total: 175ms	remaining: 9.56s
18:	learn: 88

161:	learn: 83484.3971469	total: 1.7s	remaining: 8.78s
162:	learn: 83476.6938644	total: 1.71s	remaining: 8.78s
163:	learn: 83469.5366869	total: 1.72s	remaining: 8.77s
164:	learn: 83461.7900121	total: 1.74s	remaining: 8.79s
165:	learn: 83453.6564015	total: 1.75s	remaining: 8.77s
166:	learn: 83446.6525316	total: 1.76s	remaining: 8.77s
167:	learn: 83439.8558744	total: 1.76s	remaining: 8.74s
168:	learn: 83432.7000920	total: 1.77s	remaining: 8.72s
169:	learn: 83425.4670616	total: 1.78s	remaining: 8.71s
170:	learn: 83418.2250634	total: 1.79s	remaining: 8.69s
171:	learn: 83412.1898167	total: 1.8s	remaining: 8.67s
172:	learn: 83406.4468742	total: 1.81s	remaining: 8.64s
173:	learn: 83400.3770930	total: 1.82s	remaining: 8.62s
174:	learn: 83394.7993100	total: 1.82s	remaining: 8.6s
175:	learn: 83389.2685853	total: 1.83s	remaining: 8.58s
176:	learn: 83383.2597682	total: 1.84s	remaining: 8.56s
177:	learn: 83377.1047289	total: 1.85s	remaining: 8.53s
178:	learn: 83371.2821432	total: 1.86s	remaining: 8

320:	learn: 83018.6034072	total: 3.1s	remaining: 6.57s
321:	learn: 83017.6385837	total: 3.12s	remaining: 6.57s
322:	learn: 83016.0122283	total: 3.13s	remaining: 6.56s
323:	learn: 83014.9832897	total: 3.14s	remaining: 6.56s
324:	learn: 83014.1130291	total: 3.15s	remaining: 6.55s
325:	learn: 83012.5566132	total: 3.16s	remaining: 6.54s
326:	learn: 83011.5027364	total: 3.17s	remaining: 6.52s
327:	learn: 83010.1660207	total: 3.18s	remaining: 6.51s
328:	learn: 83008.9425021	total: 3.19s	remaining: 6.5s
329:	learn: 83007.8232935	total: 3.19s	remaining: 6.48s
330:	learn: 83006.8711397	total: 3.2s	remaining: 6.47s
331:	learn: 83005.3915013	total: 3.21s	remaining: 6.46s
332:	learn: 83004.0611738	total: 3.22s	remaining: 6.44s
333:	learn: 83002.9748657	total: 3.23s	remaining: 6.43s
334:	learn: 83002.1480110	total: 3.23s	remaining: 6.42s
335:	learn: 83000.7562378	total: 3.24s	remaining: 6.41s
336:	learn: 83000.5905007	total: 3.25s	remaining: 6.38s
337:	learn: 82999.4993491	total: 3.25s	remaining: 6

487:	learn: 82868.5466009	total: 4.51s	remaining: 4.73s
488:	learn: 82867.7846116	total: 4.52s	remaining: 4.73s
489:	learn: 82866.9426601	total: 4.54s	remaining: 4.72s
490:	learn: 82866.3557204	total: 4.56s	remaining: 4.73s
491:	learn: 82865.5245378	total: 4.57s	remaining: 4.72s
492:	learn: 82864.9613123	total: 4.58s	remaining: 4.71s
493:	learn: 82864.5721060	total: 4.6s	remaining: 4.71s
494:	learn: 82864.2975322	total: 4.61s	remaining: 4.7s
495:	learn: 82863.7584102	total: 4.63s	remaining: 4.71s
496:	learn: 82862.7958834	total: 4.65s	remaining: 4.7s
497:	learn: 82862.2850547	total: 4.66s	remaining: 4.7s
498:	learn: 82861.6079544	total: 4.68s	remaining: 4.7s
499:	learn: 82860.7634608	total: 4.69s	remaining: 4.69s
500:	learn: 82860.1067607	total: 4.71s	remaining: 4.69s
501:	learn: 82859.5393406	total: 4.73s	remaining: 4.69s
502:	learn: 82859.1145414	total: 4.75s	remaining: 4.69s
503:	learn: 82858.5121313	total: 4.76s	remaining: 4.68s
504:	learn: 82857.5654320	total: 4.77s	remaining: 4.6

642:	learn: 82771.2180835	total: 5.92s	remaining: 3.28s
643:	learn: 82770.8201229	total: 5.93s	remaining: 3.28s
644:	learn: 82770.4239570	total: 5.94s	remaining: 3.27s
645:	learn: 82769.8167529	total: 5.95s	remaining: 3.26s
646:	learn: 82769.4054879	total: 5.96s	remaining: 3.25s
647:	learn: 82768.4356120	total: 5.97s	remaining: 3.24s
648:	learn: 82767.6387455	total: 5.98s	remaining: 3.23s
649:	learn: 82766.9096912	total: 5.99s	remaining: 3.22s
650:	learn: 82766.4587572	total: 6s	remaining: 3.21s
651:	learn: 82766.0414041	total: 6s	remaining: 3.2s
652:	learn: 82765.4751501	total: 6.01s	remaining: 3.19s
653:	learn: 82764.8164595	total: 6.02s	remaining: 3.18s
654:	learn: 82764.3949814	total: 6.03s	remaining: 3.17s
655:	learn: 82763.9516719	total: 6.03s	remaining: 3.16s
656:	learn: 82763.4196861	total: 6.04s	remaining: 3.15s
657:	learn: 82763.1111321	total: 6.05s	remaining: 3.14s
658:	learn: 82762.5190603	total: 6.06s	remaining: 3.13s
659:	learn: 82761.8412611	total: 6.06s	remaining: 3.12s

...

437:	learn: 82882.6025062	total: 3.67s	remaining: 4.71s
438:	learn: 82881.7432028	total: 3.68s	remaining: 4.7s
439:	learn: 82880.7559581	total: 3.69s	remaining: 4.7s
440:	learn: 82879.9148256	total: 3.7s	remaining: 4.7s
441:	learn: 82878.6979976	total: 3.72s	remaining: 4.69s
442:	learn: 82877.8467045	total: 3.73s	remaining: 4.69s
443:	learn: 82877.2985629	total: 3.74s	remaining: 4.68s
444:	learn: 82876.7362844	total: 3.75s	remaining: 4.67s
445:	learn: 82875.9935215	total: 3.76s	remaining: 4.67s
446:	learn: 82875.2721641	total: 3.76s	remaining: 4.66s
447:	learn: 82874.4957109	total: 3.77s	remaining: 4.64s
448:	learn: 82873.7126028	total: 3.78s	remaining: 4.63s
449:	learn: 82872.6078234	total: 3.79s	remaining: 4.63s
450:	learn: 82871.1971016	total: 3.79s	remaining: 4.62s
451:	learn: 82870.0539622	total: 3.8s	remaining: 4.6s
452:	learn: 82869.0262531	total: 3.8s	remaining: 4.59s
453:	learn: 82868.0931772	total: 3.81s	remaining: 4.58s
454:	learn: 82867.3437661	total: 3.82s	remaining: 4.58s

584:	learn: 82757.5322239	total: 4.87s	remaining: 3.46s
585:	learn: 82756.3898650	total: 4.88s	remaining: 3.45s
586:	learn: 82755.4786216	total: 4.89s	remaining: 3.44s
587:	learn: 82754.8327473	total: 4.91s	remaining: 3.44s
588:	learn: 82754.1544099	total: 4.92s	remaining: 3.43s
589:	learn: 82753.3194960	total: 4.92s	remaining: 3.42s
590:	learn: 82752.6996752	total: 4.93s	remaining: 3.41s
591:	learn: 82751.7772742	total: 4.94s	remaining: 3.4s
592:	learn: 82751.0518963	total: 4.95s	remaining: 3.39s
593:	learn: 82750.4160715	total: 4.95s	remaining: 3.38s
594:	learn: 82749.6551057	total: 4.96s	remaining: 3.38s
595:	learn: 82749.0565686	total: 4.96s	remaining: 3.37s
596:	learn: 82747.9413353	total: 4.97s	remaining: 3.36s
597:	learn: 82747.5242727	total: 4.98s	remaining: 3.35s
598:	learn: 82746.8933902	total: 4.99s	remaining: 3.34s
599:	learn: 82746.4449636	total: 4.99s	remaining: 3.33s
600:	learn: 82745.3512220	total: 5s	remaining: 3.32s
601:	learn: 82744.2004615	total: 5.01s	remaining: 3.

748:	learn: 82641.1462799	total: 6.08s	remaining: 2.04s
749:	learn: 82640.6827922	total: 6.09s	remaining: 2.03s
750:	learn: 82640.1966120	total: 6.1s	remaining: 2.02s
751:	learn: 82639.5948317	total: 6.11s	remaining: 2.02s
752:	learn: 82638.7257372	total: 6.12s	remaining: 2.01s
753:	learn: 82638.1620791	total: 6.13s	remaining: 2s
754:	learn: 82637.5767818	total: 6.14s	remaining: 1.99s
755:	learn: 82636.8569784	total: 6.14s	remaining: 1.98s
756:	learn: 82636.5729239	total: 6.15s	remaining: 1.98s
757:	learn: 82635.6308218	total: 6.16s	remaining: 1.97s
758:	learn: 82635.3000080	total: 6.17s	remaining: 1.96s
759:	learn: 82634.5416924	total: 6.17s	remaining: 1.95s
760:	learn: 82634.1343985	total: 6.18s	remaining: 1.94s
761:	learn: 82633.6093617	total: 6.19s	remaining: 1.93s
762:	learn: 82633.0176051	total: 6.19s	remaining: 1.92s
763:	learn: 82632.1927983	total: 6.2s	remaining: 1.92s
764:	learn: 82631.3659447	total: 6.21s	remaining: 1.91s
765:	learn: 82630.6154948	total: 6.21s	remaining: 1.9

914:	learn: 82532.4495062	total: 7.3s	remaining: 678ms
915:	learn: 82531.5739496	total: 7.31s	remaining: 670ms
916:	learn: 82530.9735019	total: 7.32s	remaining: 663ms
917:	learn: 82530.4572862	total: 7.33s	remaining: 655ms
918:	learn: 82529.8138147	total: 7.34s	remaining: 647ms
919:	learn: 82529.5589355	total: 7.35s	remaining: 639ms
920:	learn: 82529.0075270	total: 7.36s	remaining: 631ms
921:	learn: 82528.3380770	total: 7.36s	remaining: 623ms
922:	learn: 82527.8839637	total: 7.37s	remaining: 615ms
923:	learn: 82527.4792967	total: 7.38s	remaining: 607ms
924:	learn: 82526.6854457	total: 7.38s	remaining: 599ms
925:	learn: 82526.1006547	total: 7.39s	remaining: 591ms
926:	learn: 82525.6024852	total: 7.4s	remaining: 583ms
927:	learn: 82525.0620676	total: 7.41s	remaining: 575ms
928:	learn: 82524.5558283	total: 7.41s	remaining: 567ms
929:	learn: 82524.1268258	total: 7.42s	remaining: 558ms
930:	learn: 82523.0741556	total: 7.42s	remaining: 550ms
931:	learn: 82521.8636486	total: 7.43s	remaining: 

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7fa36b3b4df0>),
                              ('xgbrf',
                               XGBRFRegressor(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, g...
                                              interaction_constraints=None,
                                              max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              objective='reg:squarederror',
                                              predictor=None, random_state=None,
                                              reg_alpha=None, ...))],
                  final_estimator=LinearRegression())

In [19]:
# make predictions on the test set
stacked_pred = stacked_model.predict(X_test)

# evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, stacked_pred))
print("RMSE: ", rmse)
from sklearn.metrics import r2_score
print(r2_score(y_test, stacked_pred))

RMSE:  82770.35554401053
0.1635943656945339


In [20]:
# saving results
test_data = pd.read_csv('processed_test.csv',index_col=0)
prediction4= stacked_model.predict(test_data)

submission = pd.DataFrame({'id':test_data.index,'cltv':prediction4})

submission.to_csv('submission_stacked.csv',index=False)

# Comparison between models

In [21]:
pd.DataFrame({'cat_model':y_pred_cat.astype('int'), \
             'xgbrf_regressor':y_pred_xgbrf_regressor.astype('int'),\
              'linear':linear_model.predict(X_test).astype('int'),\
              'stacked_model':stacked_pred.astype('int'),'Actual':y_test},index=y_test.index)

,cat_model,xgbrf_regressor,linear,stacked_model,Actual
id,,,,,
48088,129005,131480,124537,129076,330168
86917,127290,131157,126807,127901,264228
59874,126762,122929,125642,126361,57720
59081,115010,111735,118581,115113,103332
62158,104359,108760,109714,105421,63948
...,...,...,...,...,...
54036,135500,132568,134735,135105,59256
5869,47909,48066,42726,47170,62808
7001,93649,97023,96611,94305,65304


Our prediction is lot closer for lower range 'cltv' score than for large values